In [1]:
import numpy as np
import pandas as pd
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error, 
    make_scorer
)

 # модели
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import (
    LinearRegression, 
    Ridge
)

from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    RandomizedSearchCV, 
    GridSearchCV
)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
df_train = pd.read_csv('data/df_train.csv', sep=';')
df_test = pd.read_csv('data/df_test.csv', sep=';')

In [4]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

In [5]:
df_train

,Id,Data,Target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591..."
...,...,...,...
7028,7028,"[6010, 4829, 6011, 6011, 6011, 6010, 6011, 601...","[4814, 5499, 5499, 5411, 5251, 4814, 5499, 549..."
7029,7029,"[4814, 5699, 5641, 5411, 6010, 6011, 4814, 601...","[6011, 5261, 6011, 5261, 4814, 5411, 4814, 601..."
7030,7030,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 601...","[6011, 6011, 6011, 6011, 6011, 6011, 6011, 601..."
7031,7031,"[4814, 4814, 5411, 6011, 6011, 4814, 4814, 481...","[4814, 6011, 6011, 5999, 5999, 6011, 6011, 601..."


In [7]:


# Преобразование датафрейма в список списков
list_of_lists = df_train['Data'].values.tolist()

# Вывод списка списков
print(len(list_of_lists))

7033


In [60]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [61]:
mapk_scorer = make_scorer(mapk, greater_is_better=True)

In [10]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Предположим, что у вас есть тренировочные данные и целевые значения
df_train = pd.read_csv('data/df_train.csv', sep=';')
df_test = pd.read_csv('data/df_test.csv', sep=';')

df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))


train_data = df_train['Data'].values.tolist()  # список списков с тренировочными данными
train_target = df_train['Target'].values.tolist()  # список списков с целевыми значениями

# Преобразование данных и целевых значений в массивы NumPy
train_data = np.array(train_data[:100])
train_target = np.array(train_target[:100])

# Определение размерности входных данных (10 чисел в последовательности)
input_dim = 10

# Создание модели LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(input_dim, 1)))  # 64 - количество скрытых нейронов
model.add(Dense(input_dim))  # Полносвязный слой для предсказания последовательности

# Компиляция модели
model.compile(loss='mse', optimizer='adam')

# Обучение модели
model.fit(train_data, train_target, epochs=10, batch_size=32)

# Предсказание последовательности
test_data = df_test['Data'].values.tolist()  # список списков с тестовыми данными
test_data = np.array(test_data[:100])
predictions = model.predict(test_data)

# Вывод предсказанных последовательностей
for prediction in predictions:
    print(prediction)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.